## CMGP: Train and Evaluation

This notebook presents the solution for training and evaluating CMGP.

The implementation of CMGP is adapted in the local `ite` library.

First, make sure that all the depends are installed.
```
pip install -r requirements.txt
pip install .
```

### Setup

First, we import all the dependencies necessary for the task.

In [ ]:
# Double check that we are using the correct interpreter.
import sys
print(sys.executable)
    
# Import depends
import ite.algs.causal_multitask_gaussian_processes.model as alg
import ite.datasets as ds
import ite.utils.numpy as utils
import numpy as np

from matplotlib import pyplot as plt

### Load the Dataset

For CGMP, we have to downsample to 1000 training items.

In [ ]:
train_ratio = 0.8
 
[Train_X, Train_T, Train_Y, Opt_Train_Y, Test_X, Test_Y] = ds.load("twins", train_ratio, downsample=1000)

### Load the model

In [ ]:
model = alg.CMGP(
    dim=dim,
    dim_outcome=dim_outcome,
    max_gp_iterations=50,
)

assert model is not None

### Run experiments

In [ ]:
plot_experiments: dict = {
    "train_sqrt_PEHE": [],
    "test_sqrt_PEHE": [],
}
    
for experiment in range(10):
    [Train_X, Train_T, Train_Y, Opt_Train_Y, Test_X, Test_Y] = ds.load(
        "twins",
        train_ratio,
        downsample=1000,
    )
    
    dim = len(Train_X[0])
    dim_outcome = Test_Y.shape[1]
 
    model = alg.CMGP(
        dim=dim,
        dim_outcome=dim_outcome,
        max_gp_iterations=50,
    )
    assert model is not None
 
    model.fit(Train_X, Train_T, Train_Y, Test_X, Test_Y)
 
    train_metrics = model.test(Train_X, Opt_Train_Y)
    assert "sqrt_PEHE" in train_metrics
    plot_experiments["train_sqrt_PEHE"].append(train_metrics["sqrt_PEHE"])

    test_metrics = model.test(Test_X, Test_Y)
    assert "sqrt_PEHE" in test_metrics
    plot_experiments["test_sqrt_PEHE"].append(test_metrics["sqrt_PEHE"])

### Plot train metrics on the test set

In [ ]:
train_sqrt_PEHE = plot_experiments["train_sqrt_PEHE"]
test_sqrt_PEHE = plot_experiments["test_sqrt_PEHE"]
 
PEHE_train_np = np.array(train_sqrt_PEHE)[~np.isnan(np.array(train_sqrt_PEHE))]
PEHE_test_np = np.array(test_sqrt_PEHE)[~np.isnan(np.array(test_sqrt_PEHE))]
 
train_ci = utils.mean_confidence_interval(PEHE_train_np)
test_ci = utils.mean_confidence_interval(PEHE_test_np)

fig, axs = plt.subplots(2)
axs[0].plot(train_sqrt_PEHE, color="r", label="train_sqrt_PEHE")
axs[0].fill_between(
    range(len(train_sqrt_PEHE)),
    train_sqrt_PEHE - train_ci[1],
    train_sqrt_PEHE + train_ci[1],
    color="r",
    alpha=0.2,
)
axs[0].legend()
         
axs[1].plot(test_sqrt_PEHE, label="test_sqrt_PEHE")
axs[1].fill_between(
    range(len(test_sqrt_PEHE)),
    test_sqrt_PEHE - test_ci[1],
    test_sqrt_PEHE + test_ci[1],
    color="b",
    alpha=0.2,
)
axs[1].legend()

### Test
Will can run inferences and get metrics directly

In [ ]:
metrics = model.test(Test_X, Test_Y)
 
metrics['sqrt_PEHE']